## mimlsvm

As described in Shen et al 2009: http://ieeexplore.ieee.org/document/5346261/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from hausdorff import hausdorff

from nltk import TextTilingTokenizer
from scipy.spatial.distance  import directed_hausdorff, pdist

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../src')
sys.path.append(src_dir)

In [3]:
%aimport data.movielens_20m_imdb
%aimport helpers.labels,helpers.neighbours, helpers.segments
%aimport utils.dataframes, utils.clusters

In [4]:
from data.movielens_20m_imdb import load_or_get_from_cache
from helpers.labels import truncate_labels
from helpers.neighbours import get_predicted_labels_from_neighbours
from helpers.segments import make_distance_matrix_for_segments,vectorize_segments

from utils.dataframes import sample_rows
from utils.clusters import k_medoids

In [5]:
INTERIM_DATA_ROOT = os.path.abspath("../../data/interim/movielens-ml20m-imdb/")
ML_ROOT = "/media/felipe/SAMSUNG/movielens/ml-20m/"
IMDB_ROOT = "/media/felipe/SAMSUNG/imdb/"

PATH_TO_MOVIES = ML_ROOT + "/movies.csv"
PATH_TO_TAG_ASSIGNMENTS = ML_ROOT + "/tags.csv"
PATH_TO_MOVIE_PLOTS = IMDB_ROOT+"/plot.list"

# CONFIGS
MAX_NB_WORDS = 300
PREPROC=None
STOP_WORDS='english'
NORM=None

MIN_LABEL_DF = 5 # like in the paper

SAMPLE_TO_NB_MEDOIDS_RATIO = 0.2 # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_KERNEL='poly'
SVM_GAMMA=0.2 # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_C= 1# not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_DEGREE=4

In [6]:
docs_df = load_or_get_from_cache(PATH_TO_MOVIES,PATH_TO_TAG_ASSIGNMENTS,PATH_TO_MOVIE_PLOTS,INTERIM_DATA_ROOT)

In [7]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
# remove this for production
docs_df = sample_rows(docs_df,500)

In [9]:
docs_df['sentences'] = docs_df['plot'].map(lambda row: sentence_tokenizer.tokenize(row))

In [10]:
docs_df['num_sentences'] = docs_df['sentences'].map( lambda sents: len(sents) )

In [11]:
docs_df.head()

,movie_id,title,unique_tags,num_users,num_unique_tags,plot,sentences,num_sentences
207,425,Blue Sky (1994),"tommy-lee-jones,affective-disorder,tony-richar...",5.0,6,It's the early 1960s. Nuclear engineer Hank Ma...,"[It's the early 1960s., Nuclear engineer Hank ...",20
1999,4630,No Holds Barred (1989),"sportprofessional'-wrestling,thomas-j-wright,h...",3.0,7,Rip is the World Wrestling Federation champion...,[Rip is the World Wrestling Federation champio...,6
3917,34542,Grizzly Man (2005),"documentary,werner-herzog,bears,biopic,eccentr...",37.0,25,A docudrama that centers on amateur grizzly be...,[A docudrama that centers on amateur grizzly b...,7
6277,119571,Teenage Dirtbag (2009),"regina-crosby,family-background,high-school",2.0,3,A popular high school girl is harassed by a de...,[A popular high school girl is harassed by a d...,2
3066,7404,Anna (1987),"nudity-topless,betamax",2.0,2,Anna is a middle-aged actress looking for work...,[Anna is a middle-aged actress looking for wor...,6


In [12]:
tok = TextTilingTokenizer(w=5, k=2)

In [13]:
def extract_segments(candidates,min_sentences=2):
    
    try:
        # we must manually insert "\n\n" because this is how 
        # texttilingtokenizer requires candidate boundaries to be 
        # represented.
        segments = tok.tokenize("\n\n".join(candidates))
    except ValueError:
        # this happens when the candidate list is too small for the 
        # text tiling tokenizer to be able to find segments. so just return
        # the original sentences.
        segments= candidates
        
    # now remove the artificially added chars
    segments = [segment.replace("\n\n"," ").strip() for segment in segments]
    
    return segments

In [14]:
docs_df['segments'] = docs_df['sentences'].map(lambda candidates: extract_segments(candidates))

In [15]:
# pickle.dump(docs_df, open(INTERIM_DATA_ROOT.rstrip('/') + "/segmented_docs_df.p", "wb"))

In [16]:
segments = docs_df['segments'].values
documents = docs_df['plot'].values
labelsets = truncate_labels(docs_df["unique_tags"].map(lambda tagstring: tagstring.split(",")).values,5)

In [17]:
# I can't put this into a pipeline because NearestNeighbors is not a normal classifier, I think
# I need to customize the pipeline object to be able to call the methods for that class.

# TFIDF_VECTORIZER = COUNT_VECTORIZER + TFIDF_TRANSFORMER
tfidf_vectorizer = TfidfVectorizer(max_features=MAX_NB_WORDS, preprocessor=PREPROC, stop_words=STOP_WORDS,norm=NORM)
# segments => k-medoids
clf = OneVsRestClassifier(SVC(kernel=SVM_KERNEL,gamma=SVM_GAMMA,C=SVM_C,degree=SVM_DEGREE),n_jobs=-1)

In [18]:
segments_train, segments_test, documents_train, documents_test, y_train, y_test = train_test_split(segments,
                                                                                                   documents,
                                                                                                   labelsets,
                                                                                                   test_size=0.25)

In [19]:
# the binarizer needs to be fit on all labels
mlb = MultiLabelBinarizer()
mlb.fit(labelsets)

y_train = mlb.transform(y_train)
y_test = mlb.transform(y_test)

## train

In [20]:
# train
tfidf_vectorizer.fit(documents_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=300, min_df=1,
        ngram_range=(1, 1), norm=None, preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [21]:
tfidf_segments_train = vectorize_segments(segments_train, tfidf_vectorizer)

In [22]:
dist_matrix_train = make_distance_matrix_for_segments(tfidf_segments_train)

In [23]:
dist_matrix_train.shape

(375, 375)

In [24]:
NB_MEDOIDS = int(len(tfidf_segments_train) * SAMPLE_TO_NB_MEDOIDS_RATIO)
medoids_indices_train = k_medoids(dist_matrix_train,NB_MEDOIDS)[0]

In [25]:
medoids = tfidf_segments_train[medoids_indices_train]

In [26]:
medoids.shape

(75,)

In [27]:
def make_train_dataset(distance_matrix, medoid_indices):
    """
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param distance_matrix: MxM matrix with pairwise distances
    :param medoid_indices: array of length N containing the indices of the medoids for each cluster
    :return: distances to medoids (MxN matrix)
    """

    return distance_matrix[:,medoid_indices]

In [28]:
medoids_indices_train

array([  7,   8,  10,  14,  15,  18,  23,  24,  36,  47,  51,  54,  55,
        62,  69,  71,  72,  74,  79,  80,  83,  84,  89,  93,  98,  99,
       100, 116, 124,  37, 144, 150, 155, 163, 171,  27, 185, 112, 188,
       197, 200,  35, 313, 211,   5, 217,  60, 220, 222, 224, 226, 231,
       235, 239, 242, 253, 254, 262, 269, 272, 279, 162, 208,   6, 176,
       282, 319, 321, 323, 227, 346, 348, 349, 362,  12])

In [29]:
X_train = make_train_dataset(dist_matrix_train,medoids_indices_train)

## test

In [30]:
# tfidf has been fit on the training set
tfidf_segments_test = vectorize_segments(segments_test, tfidf_vectorizer)

In [31]:
def make_test_dataset(source_vectorized_segments, medoid_vectorized_segments):
    """
    Calculates the distances from every source_document (reprsented by its segments) to every medoid
    document (also represented by its segments) using the hausdorff distance.
    
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param source_vectorized_segments: array of length M, where each element is a matrix with one row
        for every segment in a source document
    :param medoid_vectorized_segments: array of length N where each element is a matrix with one row
        for every segment in a medoid document
    :return: distances to medoids (MxN matrix)
    """
    
    num_test_samples = len(source_vectorized_segments)
    num_medoids = len(medoid_vectorized_segments)
    
    test_dataset = np.zeros((num_test_samples,num_medoids))    
    
    for i,source_segments in enumerate(source_vectorized_segments):
        for j,medoid_segments in enumerate(medoid_vectorized_segments):
            test_dataset[i][j] = hausdorff(source_segments.toarray(),medoid_segments.toarray())
            
    return np.array(test_dataset)
            

In [32]:
X_test = make_test_dataset(tfidf_segments_test,medoids)

## running

In [33]:
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [34]:
X_train[350]

array([ 15.71776889,  13.62023676,  10.38704136,  12.82214412,
        10.57465503,  17.73685149,  12.02562574,  11.56034855,
        12.20059657,  11.79844655,  15.49668636,  15.2526684 ,
        17.42694854,  11.57759009,  21.38586592,  12.41514748,
        15.53100654,  11.19993546,  11.17886795,  16.95841558,
        15.11934932,  11.14643094,  11.06266367,  20.69020585,
        10.04159697,  11.1355049 ,  12.91798264,  11.64691264,
        20.4509587 ,  18.68276285,  17.60797368,  12.74959792,
        11.29899783,  12.83522488,  11.83537744,  21.96676197,
        10.17513773,  17.24596074,  14.28649236,  15.78405279,
        23.0176805 ,  13.81791546,  10.02726717,  11.30980954,
        23.04659591,  10.44440839,  23.65249612,  12.96481437,
        11.55881968,  12.97768306,  10.69007906,  20.47765866,
        14.95101036,  12.81259041,  27.77782411,  14.70965538,
        11.55701458,  27.1245738 ,  11.22522191,  12.48769948,
        16.43166835,  14.29239545,  12.97905173,   9.89

In [35]:
clf.fit(X_train,y_train)

OneVsRestClassifier(estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=4, gamma=0.2, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-1)

In [36]:
y_preds = clf.predict(X_test)
y_trues = y_test

In [37]:
y_preds[0].shape

(200,)

In [38]:
np.allclose(y_preds[77],np.zeros(y_preds.shape[1]))

True

In [39]:
f1_score(y_trues,y_preds,average='micro')

0.0